In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd

browser = Browser('chrome')

In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]"
browser.visit(url)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)


In [4]:
cookie = browser.find_by_id('btn-cookie-allow').click()
# cookie.click()

In [5]:
# This is for loading more products 
max_iteration = 1
iteration = 0

while iteration<= max_iteration:
    # Extract the HTML of the current page
    html = browser.html
    
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')
    
    all_links = soup.find_all("div", class_ = 'coveo-product-items')

    links = []
    
    for link in all_links:
        results = link.a['href']
        links.append(results)
        
    # Find the element by ID
    element = soup.find(id='loadMore')
    
    if element is None:
        # Element not found, exit the loop
        break
        
    # Wait for 2 seconds
    time.sleep(1)
    
    
        
    # Click on the element
    browser.find_by_id('loadMore').click()
    
    
    
    iteration += 1
    
    print(f'Iteration #: {iteration}')
    


Iteration #: 1
Iteration #: 2


In [6]:
len(links)

48

In [7]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []
taste_list = []

max_iteration2 = 4
iteration2 = 0


for record in links:
    if iteration2 >= max_iteration2:
        break
    url = record
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)
#     browser.find_by_css('my_store_close_button').click()
    

    old_price = soup.find('span', class_ = 'old-price')
    old = old_price.find('span', class_ = 'price')
    regular_price.append(old.find('span', class_ = 'price').text)

    on_sale_price = soup.find('span', class_ = 'special-price')
    new_price = on_sale_price.find('span', class_ = 'price')
    sale_price.append(new_price.find('span', class_ = 'price').text)

    name.append(soup.find('div', class_ = 'page-title-wrapper').text)
    try:
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
    except AttributeError:
        rating.append('N/A')
    try:
        rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
    except AttributeError:
        rating.append('N/A')
    reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
    size.append(soup.find('div', class_ = 'lcbo-product-size').text)
    description.append(soup.find('div', class_ = 'testing_note').text)

    categories_all = soup.find('div', class_ = 'breadcrumbs')
    categories = categories_all.find_all('li', class_ = 'item')
    category_list = []

    for category_record in categories:
        category_list.append(category_record.text)
    category.append(category_list[-2])

    moredetail = soup.find('div', class_ = 'moredetail')
    all_details = moredetail.find_all('li')
    details = {}

    for detail_record in all_details:
        details_value = detail_record.find('div', class_ = 'value').text
        details_name = detail_record.find('div', class_ = 'label').text
        details[details_name] = details_value

    details_list.append(details)
#     try:
#         moretaste = soup.find('div', class_ = 'foodParings pip-info')
#         all_taste = moretaste.find_all('li')
#         taste = {}

#         for taste_record in all_taste:
#             taste_value1 = taste_record.find('div', class_ = 'value').text
#             taste_name = taste_record.find('div', class_ = 'label').text
#             taste_value = taste_value1.get('aria-label')
#             taste[taste_name] = taste_value
#         taste_list.append(taste)
#     except AttributeError:
#         taste_list.append('N/A')
    
    print(iteration2)
    iteration2 += 1



0
1
2
3


In [9]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category
}

# regular_price = [] ok
# sale_price = [] ok
# name = [] ok
# rating = [] ok
# reviews = [] ok
# size = [] ok
# description = [] ok
# category = [] ok 

# details = {}
# Add the details dictionary to the data dictionary
data['Details'] = details_list
# data['Taste'] = taste_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

details_df = pd.json_normalize(df['Details'])
# taste_df = pd.json_normalize(df['Taste'])

df = pd.concat([df, details_df], axis = 1)
# df2 = pd.concat([df1, taste_df], axis = 1)

df.drop('Details', axis = 1, inplace = True)
# df2.drop('Taste', axis = 1, inplace = True)

# Display the dataframe Test
df.head(20)

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Alcohol/Vol,Made In,By,Sugar Content,Varietal
0,Josh Cellars Pinot Grigio,$19.95,$17.95,4.5,(17),750 ml bottle,Josh Cellars is a renowned brand created to ce...,White Wine,12.5%,"California, United States",Josh Cellars,3 g/L,Pinot Grigio
1,Ruffino Prosecco Rose DOC,$20.00,$18.00,5.0,(2),750 ml bottle,Ruffino has been at the forefront of Italian w...,Rosé & Red Sparkling Wine,11%,"Friuli, Italy",Ruffino Srl Wines,21 g/L,Sparkling
2,Nude Pink Lemonade,$2.85,$2.70,3.9,(16),355 ml can,"Nude's newest seltzer is an all-natural, prett...",Seltzers and Sodas,5%,"Ontario, Canada",Nude,NaN,NaN
3,Meukow Cognac VS,$65.95,$61.95,4.7,(3),750 ml bottle,Awarded in 2015 both Best Cognac VS and Double...,V.S. Cognac,40%,"Cognac, France",Meukow,NaN,NaN


In [ ]:
print(taste_list)